In [ ]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):
        return re.sub(r"\s*\(.*?\)", "", text)
    return text

In [ ]:
# Get the rows which Number of Companions is 4-6
companion = data[data["Number of Companions"] == "4-6"]

In [3]:
# Split the values into a single items
companion["Preferred Spa and Wellness Activities"] = companion["Preferred Spa and Wellness Activities"].apply(remove_parentheses).str.split(",")

# Convert the items into a transactional data
transactions = companion["Preferred Spa and Wellness Activities"].explode().str.strip().reset_index()
transactions_encoded = transactions.pivot_table(index="index", columns="Preferred Spa and Wellness Activities", aggfunc=lambda x: 1, fill_value=0)

C:\Users\NJ\AppData\Local\Temp\ipykernel_11976\586481822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companion["Preferred Spa and Wellness Activities"] = companion["Preferred Spa and Wellness Activities"].apply(remove_parentheses).str.split(",")


In [4]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.3, use_colnames=True)# threshold 30%
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\NJ\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
1,(Spa Treatment),(Massage Therapy),0.65625,0.954545,1.053292
2,"(Sauna and Steam Rooms, Spa Treatment)",(Massage Therapy),0.46875,0.937500,1.034483
0,(Sauna and Steam Rooms),(Massage Therapy),0.56250,0.857143,0.945813
3,"(Sauna and Steam Rooms, Massage Therapy)",(Spa Treatment),0.46875,0.833333,1.212121
